<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Low-Level-Programming" data-toc-modified-id="Low-Level-Programming-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Low Level Programming</a></span><ul class="toc-item"><li><span><a href="#unsafe.Sizeof" data-toc-modified-id="unsafe.Sizeof-13.1"><span class="toc-item-num">13.1&nbsp;&nbsp;</span>unsafe.Sizeof</a></span></li><li><span><a href="#unsafe.Pointer" data-toc-modified-id="unsafe.Pointer-13.2"><span class="toc-item-num">13.2&nbsp;&nbsp;</span>unsafe.Pointer</a></span></li><li><span><a href="#Example:-Deep-Equivalence" data-toc-modified-id="Example:-Deep-Equivalence-13.3"><span class="toc-item-num">13.3&nbsp;&nbsp;</span>Example: Deep Equivalence</a></span></li><li><span><a href="#Calling-C-Code-with-cgo" data-toc-modified-id="Calling-C-Code-with-cgo-13.4"><span class="toc-item-num">13.4&nbsp;&nbsp;</span>Calling C Code with cgo</a></span></li></ul></li></ul></div>

# Low Level Programming

## unsafe.Sizeof
* The unsafe.Sizeof function reports the size in bytes of the representation of its operand
  * may be an expression of any type

In [48]:
import "unsafe"
%%
fmt.Println(unsafe.Sizeof(float64(0))) // “8”

8


In [28]:
var x struct {
  a bool
  b int16
  c []int
}
%%
fmt.Println(unsafe.Sizeof(x),unsafe.Sizeof(x.a),unsafe.Sizeof(x.b),unsafe.Sizeof(x.c))
fmt.Println(unsafe.Alignof(x),unsafe.Alignof(x.a),unsafe.Alignof(x.b),unsafe.Alignof(x.c))
fmt.Println(unsafe.Offsetof(x.a),unsafe.Offsetof(x.b),unsafe.Offsetof(x.c))

32 1 2 24
8 1 2 8
0 2 8


## unsafe.Pointer

In [67]:
//package math
func Float64bits(f float64) uint64 { 
    return *(*uint64)(unsafe.Pointer(&f)) 
}
%%
fmt.Printf("%#016x\n", Float64bits(1.0)) // 0x3ff0000000000000
fmt.Printf("%16x\n",1.0)

0x3ff0000000000000
         0x1p+00


In [68]:
import (
        "fmt"
        "unsafe"
)

func main() {
        //!+main
        var x struct {
                a bool
                b int16
                c []int
        }

        // equivalent to pb := &x.b
        pb := (*int16)(unsafe.Pointer(
                uintptr(unsafe.Pointer(&x)) + unsafe.Offsetof(x.b)))
        *pb = 42

        fmt.Println(x.b) // "42"
    
        tmp := uintptr(unsafe.Pointer(&x)) + unsafe.Offsetof(x.b)
        pb  = (*int16)(unsafe.Pointer(tmp))
        *pb = 43
        fmt.Println(x.b) 
        //!-main
}
/*
//!+wrong
        // NOTE: subtly incorrect!
        tmp := uintptr(unsafe.Pointer(&x)) + unsafe.Offsetof(x.b)
        pb := (*int16)(unsafe.Pointer(tmp))
        *pb = 42
//!-wrong
*/


42
43


<code>
pT := uintptr(unsafe.Pointer(new(T))) // NOTE: wrong!
</code>

## Example: Deep Equivalence

In [15]:
func TestSplit( t *testing.T) {
   got := strings.Split("a:b:c", ":")
   want := []string{"a","b","c"}
   if !reflect.DeepEqual(got, want) { /* … */ }
}
%%
var a,b []string = nil, []string{}
fmt.Println(reflect.DeepEqual(a,b)) // “false”
var c,d map[string]int = nil, make(map[string]int)
fmt.Println(reflect.DeepEqual(c,d)) // “false”

false
false


In [32]:
// Package equal provides a deep equivalence relation for arbitrary values.
//package equal

import (
        "reflect"
        "unsafe"
)

//!+
func equal(x, y reflect.Value, seen map[comparison]bool) bool {
        if !x.IsValid() || !y.IsValid() {
                return x.IsValid() == y.IsValid()
        }
        if x.Type() != y.Type() {
                return false
        }

        // ...cycle check omitted (shown later)...
//!-
        //!+cyclecheck
        // cycle check
        if x.CanAddr() && y.CanAddr() {
                xptr := unsafe.Pointer(x.UnsafeAddr())
                yptr := unsafe.Pointer(y.UnsafeAddr())
                if xptr == yptr {
                        return true // identical references
                }
                c := comparison{xptr, yptr, x.Type()}
                if seen[c] {
                        return true // already seen
                }
                seen[c] = true
        }
        //!-cyclecheck
        //!+
        switch x.Kind() {
        case reflect.Bool:
                return x.Bool() == y.Bool()
        case reflect.String:
                return x.String() == y.String()

        // ...numeric cases omitted for brevity...

        //!-
        case reflect.Int, reflect.Int8, reflect.Int16, reflect.Int32,
                reflect.Int64:
                return x.Int() == y.Int()

        case reflect.Uint, reflect.Uint8, reflect.Uint16, reflect.Uint32,
                reflect.Uint64, reflect.Uintptr:
                return x.Uint() == y.Uint()

        case reflect.Float32, reflect.Float64:
                return x.Float() == y.Float()

        case reflect.Complex64, reflect.Complex128:
            return x.Complex() == y.Complex()
        //!+
        case reflect.Chan, reflect.UnsafePointer, reflect.Func:
                return x.Pointer() == y.Pointer()

        case reflect.Ptr, reflect.Interface:
                return equal(x.Elem(), y.Elem(), seen)

        case reflect.Array, reflect.Slice:
                if x.Len() != y.Len() {
                        return false
                }
                for i := 0; i < x.Len(); i++ {
                        if !equal(x.Index(i), y.Index(i), seen) {
                                return false
                        }
                }
                return true

        // ...struct and map cases omitted for brevity...
  //!-
        case reflect.Struct:
                for i, n := 0, x.NumField(); i < n; i++ {
                        if !equal(x.Field(i), y.Field(i), seen) {
                                return false
                        }
                }
                return true

        case reflect.Map:
                if x.Len() != y.Len() {
                        return false
                }
                for _, k := range x.MapKeys() {
                        if !equal(x.MapIndex(k), y.MapIndex(k), seen) {
                                return false
                        }
                }
                return true
//!+
        }
        panic("unreachable")
}

//!-

//!+comparison
// Equal reports whether x and y are deeply equal.
//!-comparison
//
// Map keys are always compared with ==, not deeply.
// (This matters for keys containing pointers or interfaces.)
//!+comparison
func Equal(x, y interface{}) bool {
        seen := make(map[comparison]bool)
        return equal(reflect.ValueOf(x), reflect.ValueOf(y), seen)
}


type comparison struct {
        x, y unsafe.Pointer
        t    reflect.Type
}

//!-comparison


In [33]:
func Example_equal() {
        //!+
        fmt.Println(Equal([]int{1, 2, 3}, []int{1, 2, 3}))        // "true"
        fmt.Println(Equal([]string{"foo"}, []string{"bar"}))      // "false"
        fmt.Println(Equal([]string(nil), []string{}))             // "true"
        fmt.Println(Equal(map[string]int(nil), map[string]int{})) // "true"
        //!-

        // Output:
        // true
        // false
        // true
        // true
}

In [34]:
%%
Example_equal()

true
false
true
true


In [35]:
func Example_equalCycle() {
        //!+cycle
        // Circular linked lists a -> b -> a and c -> c.
        type link struct {
                value string
                tail  *link
        }
        a, b, c := &link{value: "a"}, &link{value: "b"}, &link{value: "c"}
        a.tail, b.tail, c.tail = b, a, c
        fmt.Println(Equal(a, a)) // "true"
        fmt.Println(Equal(b, b)) // "true"
        fmt.Println(Equal(c, c)) // "true"
        fmt.Println(Equal(a, b)) // "false"
        fmt.Println(Equal(a, c)) // "false"
        //!-cycle

        // Output:
        // true
        // true
        // true
        // false
        // false
}

In [36]:
%%
Example_equalCycle() 

true
true
true
false
false


## Calling C Code with cgo

In [ ]:
// typedef int (*intFunc) ();
//
// int
// bridge_int_func(intFunc f)
// {
//		return f();
// }
//
// int fortytwo()
// {
//	    return 42;
// }
import "C"
import "fmt"

func main() {
	f := C.intFunc(C.fortytwo)
	fmt.Println(int(C.bridge_int_func(f)))
	// Output: 42
}

<code>
//!+
/* This file is gopl.io/ch13/bzip/bzip2.c,         */
/* a simple wrapper for libbzip2 suitable for cgo. */
#include <bzlib.h>

int bz2compress(bz_stream *s, int action,
                char *in, unsigned *inlen, char *out, unsigned *outlen) {
  s->next_in = in;
  s->avail_in = *inlen;
  s->next_out = out;
  s->avail_out = *outlen;
  int r = BZ2_bzCompress(s, action);
  *inlen -= s->avail_in;
  *outlen -= s->avail_out;
  s->next_in = s->next_out = NULL;
  return r;
}

//!-
</code>

In [ ]:
// Package bzip provides a writer that uses bzip2 compression (bzip.org).
//package bzip

/*
#cgo CFLAGS: -I/usr/include
#cgo LDFLAGS: -L/usr/lib -lbz2
#include <bzlib.h>
#include <stdlib.h>
bz_stream* bz2alloc() { return calloc(1, sizeof(bz_stream)); }
int bz2compress(bz_stream *s, int action,
                char *in, unsigned *inlen, char *out, unsigned *outlen);
void bz2free(bz_stream* s) { free(s); }
*/
import "C"

import (
        "io"
        "unsafe"
)

type writer struct {
        w      io.Writer // underlying output stream
        stream *C.bz_stream
        outbuf [64 * 1024]byte
}

// NewWriter returns a writer for bzip2-compressed streams.
func NewWriter(out io.Writer) io.WriteCloser {
        const blockSize = 9
        const verbosity = 0
        const workFactor = 30
        w := &writer{w: out, stream: C.bz2alloc()}
        C.BZ2_bzCompressInit(w.stream, blockSize, verbosity, workFactor)
        return w
}
//!-

//!+write
func (w *writer) Write(data []byte) (int, error) {
        if w.stream == nil {
                panic("closed")
        }
        var total int // uncompressed bytes written

        for len(data) > 0 {
                inlen, outlen := C.uint(len(data)), C.uint(cap(w.outbuf))
                C.bz2compress(w.stream, C.BZ_RUN,
                        (*C.char)(unsafe.Pointer(&data[0])), &inlen,
                        (*C.char)(unsafe.Pointer(&w.outbuf)), &outlen)
                total += int(inlen)
                data = data[inlen:]
                if _, err := w.w.Write(w.outbuf[:outlen]); err != nil {
                        return total, err
                }
        }
        return total, nil
}

//!-write

//!+close
// Close flushes the compressed data and closes the stream.
// It does not close the underlying io.Writer.
func (w *writer) Close() error {
        if w.stream == nil {
                panic("closed")
        }
        defer func() {
                C.BZ2_bzCompressEnd(w.stream)
                C.bz2free(w.stream)
                w.stream = nil
 }()
        for {
                inlen, outlen := C.uint(0), C.uint(cap(w.outbuf))
                r := C.bz2compress(w.stream, C.BZ_FINISH, nil, &inlen,
                        (*C.char)(unsafe.Pointer(&w.outbuf)), &outlen)
                if _, err := w.w.Write(w.outbuf[:outlen]); err != nil {
                        return err
                }
                if r == C.BZ_STREAM_END {
                        return nil
                }
        }
}

//!-close

In [41]:
//bzipper
import (
        "io"
        "log"
        "os"

        "gopl.io/ch13/bzip"
)

func main() {
        w := bzip.NewWriter(os.Stdout)
        if _, err := io.Copy(w, os.Stdin); err != nil {
                log.Fatalf("bzipper: %v\n", err)
        }
        if err := w.Close(); err != nil {
                log.Fatalf("bzipper: close: %v\n", err)
        }
}


^C
signal: interrupt


In [42]:
!go build gopl.io/ch13/bzipper

# gopl.io/ch13/bzipper
ld: warning: '/private/var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/go-link-2835667321/go.o' has malformed LC_DYSYMTAB, expected 51 undefined symbols to start at index 3063, found 62 undefined symbols starting at index 15


In [43]:
!wc -c < /usr/share/dict/words

 2493885


In [44]:
!sha3sum < /usr/share/dict/words 

df33787900e261a4c98a8863dbe3455aa3de9182511c6697b9208bb9  -


In [45]:
!./bzipper < /usr/share/dict/words | wc -c

  857578


In [46]:
!./bzipper < /usr/share/dict/words | bunzip2 | sha3sum

df33787900e261a4c98a8863dbe3455aa3de9182511c6697b9208bb9  -
